In [34]:
import numpy as np
import json
from typing import List
import os, os.path
from collections import defaultdict

Got a dataset from Slay The Spire official discord where Baalor posted data about around 700 of his runs in 2022.

### Can we deterministically say what's the state of the deck at any floor?


In [45]:
IRONCLAD_CARDS = ["Anger", "Body Slam", "Clash", "Cleave", "Clothesline", "Headbutt", "Heavy Blade", "Iron Wave", "Perfected Strike", "Pommel Strike", "Sword Boomerang", "Thunderclap", "Twin Strike", "Wild Strike", "Blood for Blood", "Carnage", "Dropkick", "Hemokinesis", "Pummel", "Rampage", "Reckless Charge", "Searing Blow", "Sever Soul", "Uppercut", "Whirlwind", "Bludgeon", "Feed", "Fiend Fire", "Immolate", "Reaper", "Armaments", "Flex", "Havoc", "Shrug It Off", "True Grit", "Warcry", "Battle Trance", "Bloodletting", "Burning Pact", "Disarm", "Dual Wield", "Entrench", "Flame Barrier", "Ghostly Armor", "Infernal Blade", "Intimidate", "Power Through", "Rage", "Second Wind", "Seeing Red", "Sentinel", "Shockwave", "Spot Weakness", "Double Tap", "Exhume", "Impervious", "Limit Break", "Offering", "Combust", "Dark Embrace", "Evolve", "Feel No Pain", "Fire Breathing", "Inflame", "Metallicize", "Rupture", "Barricade", "Berserk", "Brutality", "Corruption", "Demon Form", "Juggernaut", ]
COLORLESS_CARDS = ["Dramatic Entrance", "Flash of Steel", "Mind Blast", "Swift Strike", "Hand of Greed", "Bite", "Expunger", "Ritual Dagger", "Shiv", "Smite", "Through Violence", "Bandage Up", "Blind", "Dark Shackles", "Deep Breath", "Discovery", "Enlightenment", "Finesse", "Forethought", "Good Instincts", "Impatience", "Jack of All Trades", "Madness", "Panacea", "Panic Button", "Purity", "Trip", "Apotheosis", "Chrysalis", "Master of Strategy", "Metamorphosis", "Secret Technique", "Secret Weapon", "The Bomb", "Thinking Ahead", "Transmutation", "Violence", "Apparition", "Beta", "Insight", "J.A.X.", "Miracle", "Safety", "Magnetism", "Mayhem", "Panache", "Sadistic Nature", "Omega", ]

def is_a_card(card : str):
    name = card.split("+")[0]
    return (name in IRONCLAD_CARDS) or (name in COLORLESS_CARDS)
    
def add_card(deck : List[str], card : str):
    if is_a_card(card):
        deck.append(card)

def remove_card(deck : List[str], card : str):
    if card in deck:
        deck.remove(card)
    else:
        assert False, f"{card} not in {deck}"

def upgrade_card(deck : List[str], card : str):
    remove_card(deck, card)
    if "+" in card:
        name, n_upgrades = card.split("+")
        n_upgrades = int(n_upgrades)
    else:
        name = card
        n_upgrades = 0
    card = f"{name}+{n_upgrades+1}"
    add_card(deck, card)

def rebuild_deck(data):
    assert data["is_ascension_mode"]
    assert data["character_chosen"] == "IRONCLAD"
    assert data["victory"]
    assert data["ascension_level"] == 20
    assert len(data["path_per_floor"]) == 57

    deck = ["Defend_R"]*4 + ["Strike_R"]*5 + ["Bash", "AscendersBane"]

    for card in data["neow_bonus_log"]["cardsObtained"]:
        add_card(deck, card)
    for card in data["neow_bonus_log"]["cardsUpgraded"]:
        upgrade_card(deck, card)
    for card in data["neow_bonus_log"]["cardsRemoved"]:
        remove_card(deck, card)
    for card in data["neow_bonus_log"]["cardsTransformed"]:
        remove_card(deck, card)

    node_to_index = defaultdict(lambda: 0)

    floor = 0
    for node in data["path_per_floor"]:
        floor += 1
        index = node_to_index[node]
        node_to_index[node] += 1

        if (node == "M") or (node == "E") or (node == "B"):
            if floor not in [50, 51, 56]:
                got_at_least_one_reward = False
                for card_choice in data["card_choices"]:
                    if card_choice["floor"] == floor:
                        got_at_least_one_reward = True
                        add_card(deck, card_choice["picked"])
        elif node == "?":
            node_data = data["event_choices"][index]
            assert node_data["floor"] == floor
            if "cards_obtained" in node_data:
                for card in node_data["cards_obtained"]:
                    add_card(deck, card)
            if "cards_upgraded" in node_data:
                for card in node_data["cards_upgraded"]:
                    upgrade_card(deck, card)
            if "cards_removed" in node_data:
                for card in node_data["cards_removed"]:
                    remove_card(deck, card)
            if "cards_transformed" in node_data:
                for card in node_data["cards_transformed"]:
                    remove_card(deck, card)
        elif node == "R":
            node_data = data["campfire_choices"][index]
            assert node_data["key"] in ["REST", "SMITH", "RECALL", "DIG", "LIFT"]
            assert node_data["floor"] == floor
            if node_data["key"] == "SMITH":
                upgrade_card(deck, node_data["data"])
        elif node == "T":
            pass
        elif node == "$":
            for purged_card, purged_floor in zip(data["items_purged"], data["items_purged_floors"]):
                if purged_floor == floor:
                    remove_card(deck, purged_card)
            for purchase_card, purchase_floor in zip(data["items_purchased"], data["item_purchase_floors"]):
                if purchase_floor == floor:
                    add_card(deck, purchase_card)
        elif node is None:
            pass
        else:
            assert False, node

    master_deck = set(data["master_deck"])
    deck = set(deck)
    assert deck == master_deck, (master_deck.difference(deck), deck.difference(master_deck))


In [46]:
directory = "../Baalor400/Wins 201-400/IRONCLAD"
files = os.listdir(directory)

for filename in files:
    data = json.load(open(os.path.join(directory, filename), "r"))
    try:
        rebuild_deck(data)
    except Exception as e:
        print(filename)
        json.dump(data, open("./example_ironclad.run", "w"), indent=4)
        raise e



1657397784.run


AssertionError: ({'Bash+1', 'Strike_R+1'}, {'Bash', 'Strike_R'})